In [1]:
library(SingleCellExperiment)
library(devtools)
library(scAnnoX)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats

Warning message:
"package 'matrixStats' was built under R version 4.3.2"

Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, 

In [2]:
# A preprocessed dataset of type Seurat.
test.obj <- readRDS('../data/GSE81252.test.obj')
ref.obj <- readRDS('../data/GSE81252.ref.obj')

### Only one single cell annotation tool is invoked

In [3]:
# Using SingleR as an example.
pred.obj <- autoAnnoTools(
    test.obj,
    ref.obj = ref.obj,
    ref.ctype = 'CellType',
    marker.lst = marker.lst,
    method = 'SingleR',
    select.marker = 'Seurat',
    top.k = 30,
    strategy = 'refernce-based'
) 
head(pred.obj)

,orig.ident,nCount_RNA,nFeature_RNA,cell_id,experiment,assignment_LB,CellType,SingleR
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
A1_he2,A1,29630.19,7229,A1_he2,he2,HE,HE,HE
A1_huvec,A1,30603.64,7374,A1_huvec,huvec,EC,EC,EC
A10_he1,A10,26921.42,6185,A10_he1,he1,HE,HE,HE
A10_lb2,A10,21383.21,4296,A10_lb2,lb2,EC-LB,EC-LB,EC-LB
A10_lb5,A10,22637.05,5307,A10_lb5,lb5,MC-LB,MC-LB,MC-LB
A11_he1,A11,16551.88,3725,A11_he1,he1,HE,HE,HE-LB
A11_lb5,A11,24529.42,5102,A11_lb5,lb5,EC-LB,EC-LB,EC-LB
A12_he1,A12,24989.38,5695,A12_he1,he1,HE,HE,HE
A12_huvec,A12,32676.21,7801,A12_huvec,huvec,EC,EC,EC


In [4]:
# Using SCINA as an example.
Idents(ref.obj) <- ref.obj$CellType
marker.lst <- findMarkerToolsForSc(ref.obj, to.list = TRUE, top.k = 30)
pred.obj <- autoAnnoTools(
    test.obj,
    ref.obj = ref.obj,
    ref.ctype = 'CellType',
    marker.lst = marker.lst,
    method = 'SCINA',
    select.marker = 'Seurat',
    top.k = 30,
    strategy = 'marker-based'
) 
head(pred.obj)

ERROR: Error in findMarkerToolsForSc(ref.obj, to.list = TRUE, top.k = 30): could not find function "findMarkerToolsForSc"


In [ ]:
a <- autoAnnoResult(pred.obj)
a

Warning message:
"The following arguments are not used: drop"


### Multiple single-cell annotation tools are invoked